<a href="https://colab.research.google.com/github/hongdnn/unsloth_finetuning_demo/blob/main/UnslothFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
  !pip install unsloth
else:
  # Do this only in Colab notebooks! Otherwise use pip install unsloth
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
  !pip install --no-deps unsloth

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length= max_seq_length,
    dtype= dtype,
    load_in_4bit= load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [6]:
model = FastLanguageModel.get_peft_model(
    model,  # The base pre-trained model that you want to fine-tune using PEFT (Parameter-Efficient Fine-Tuning) techniques.

    # r: The LoRA rank, which determines the number of trainable parameters in the low-rank adapters.
    # A higher value (e.g., 128) gives the adapter more capacity to learn task-specific nuances,
    # while lower values (e.g., 8, 16, 32, 64) might be sufficient for simpler tasks.
    r = 128,  # Choose any number > 0. Suggested values are 8, 16, 32, 64, 128.

    # target_modules: A list of module names in the model where the LoRA adapters should be applied.
    # For transformer-based models, these typically include:
    # - "q_proj", "k_proj", "v_proj", "o_proj": The projection layers in the multi-head attention mechanism.
    # - "gate_proj", "up_proj", "down_proj": Additional projection layers used in various architectures or gating mechanisms.
    # - "embed_tokens", "lm_head": Typically included for continual pretraining or when modifying the embedding and output layers.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"],  # Add these for continual pretraining if needed.

    # lora_alpha: A scaling factor applied to the LoRA weights.
    # This factor adjusts the influence of the LoRA adapters relative to the original model weights.
    lora_alpha = 32,

    # lora_dropout: Dropout rate for the LoRA adapters.
    # A dropout of 0 means no dropout is applied, which is the optimized setting in this configuration.
    lora_dropout = 0,  # Supports any value, but 0 is optimized.

    # bias: Configures the use of bias parameters in the adapters.
    # Setting this to "none" means no additional bias terms are introduced, which simplifies the model.
    bias = "none",  # Supports any value, but "none" is optimized.

    # use_gradient_checkpointing: A technique to reduce memory usage by trading compute for memory.
    # The special "unsloth" setting here is an optimized mode that reportedly reduces VRAM usage by 30%
    # and allows for twice as large batch sizes, which is particularly useful for very long context lengths.
    use_gradient_checkpointing = "unsloth",  # Use True or "unsloth" for very long context scenarios.

    # random_state: Sets a seed for the random number generator to ensure reproducibility in training.
    random_state = 3407,

    # use_rslora: Activates Rank Stabilized LoRA (RS-LoRA), an enhanced version designed to improve stability during training.
    use_rslora = True,  # Enables the use of rank stabilized LoRA.

    # loftq_config: Configuration for LoftQ, another parameter-efficient fine-tuning technique.
    # Here it is set to None, meaning that LoftQ is not applied in this configuration.
    loftq_config = None,  # LoftQ is not used in this case.
)


Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [12]:
# in case load_dataset cause error
#pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.7 requires msgspec, which is not installed.
unsloth-zoo 2025.5.7 requires tyro, which is not installed.
unsloth 2025.5.6 requires tyro, which is not installed.
unsloth-zoo 2025.5.7 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
unsloth-zoo 2025.5.7 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.

In [1]:
from datasets import load_dataset
squad = load_dataset("rajpurkar/squad", split="train[:2000]")
squad = squad.train_test_split(test_size=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [7]:
text = squad['train'][0]
text

{'id': '56cf5bfcaab44d1400b89114',
 'title': 'Frédéric_Chopin',
 'context': "Fryderyk's father, Nicolas Chopin, was a Frenchman from Lorraine who had emigrated to Poland in 1787 at the age of sixteen. Nicolas tutored children of the Polish aristocracy, and in 1806 married Justyna Krzyżanowska, a poor relative of the Skarbeks, one of the families for whom he worked. Fryderyk was baptized on Easter Sunday, 23 April 1810, in the same church where his parents had married, in Brochów. His eighteen-year-old godfather, for whom he was named, was Fryderyk Skarbek, a pupil of Nicolas Chopin. Fryderyk was the couple's second child and only son; he had an elder sister, Ludwika (1807–55), and two younger sisters, Izabela (1811–81) and Emilia (1812–27). Nicolas was devoted to his adopted homeland, and insisted on the use of the Polish language in the household.",
 'question': "What is the name of Chopin's godfather?",
 'answers': {'text': ['Fryderyk Skarbek'], 'answer_start': [468]}}

In [8]:
from datasets import load_dataset
from unsloth import UnslothTrainer, UnslothTrainingArguments, FastLanguageModel
from transformers import DataCollatorForLanguageModeling

# Preprocess
def preprocess_function(examples):
    contexts = examples["context"]
    questions = examples["question"]
    answers = [ans["text"][0] for ans in examples["answers"]]
    inputs = [f"Context: {context} Question: {question}" for context, question in zip(contexts, questions)]
    return {"input_text": inputs, "target_text": answers}

squad_processed = squad.map(preprocess_function, batched=True, num_proc=8, remove_columns=["id", "title", "context", "question", "answers"])

# Formatting with stop token
def formatting_func(example):
    return f"{example['input_text']} Answer: {example['target_text']}<|eot|>"

Map (num_proc=8):   0%|          | 0/1600 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/400 [00:00<?, ? examples/s]

In [9]:
squad_processed["train"][0]

{'input_text': "Context: Fryderyk's father, Nicolas Chopin, was a Frenchman from Lorraine who had emigrated to Poland in 1787 at the age of sixteen. Nicolas tutored children of the Polish aristocracy, and in 1806 married Justyna Krzyżanowska, a poor relative of the Skarbeks, one of the families for whom he worked. Fryderyk was baptized on Easter Sunday, 23 April 1810, in the same church where his parents had married, in Brochów. His eighteen-year-old godfather, for whom he was named, was Fryderyk Skarbek, a pupil of Nicolas Chopin. Fryderyk was the couple's second child and only son; he had an elder sister, Ludwika (1807–55), and two younger sisters, Izabela (1811–81) and Emilia (1812–27). Nicolas was devoted to his adopted homeland, and insisted on the use of the Polish language in the household. Question: What is the name of Chopin's godfather?",
 'target_text': 'Fryderyk Skarbek'}

In [10]:
formatting_func(squad_processed["train"][1])

'Context: Her debut single, "Crazy in Love" was named VH1\'s "Greatest Song of the 2000s", NME\'s "Best Track of the 00s" and "Pop Song of the Century", considered by Rolling Stone to be one of the 500 greatest songs of all time, earned two Grammy Awards and is one of the best-selling singles of all time at around 8 million copies. The music video for "Single Ladies (Put a Ring on It)", which achieved fame for its intricate choreography and its deployment of jazz hands, was credited by the Toronto Star as having started the "first major dance craze of both the new millennium and the Internet", triggering a number of parodies of the dance choreography and a legion of amateur imitators on YouTube. In 2013, Drake released a single titled "Girls Love Beyoncé", which featured an interpolation from Destiny Child\'s "Say My Name" and discussed his relationship with women. In January 2012, research scientist Bryan Lessard named Scaptia beyonceae, a species of horse fly found in Northern Queens

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #not using mask language

# Trainer
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=squad_processed["train"],
    dataset_text_field="input_text",
    formatting_func=formatting_func,
    max_seq_length=512,
    data_collator=data_collator,
    args=UnslothTrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_ratio=0.1,
        num_train_epochs=3,
        learning_rate=5e-5,
        embedding_learning_rate=5e-6,
        fp16 = not is_bfloat16_supported(), # Use 16-bit floating point if bfloat16 isn't supported.
        bf16 = is_bfloat16_supported(),     #use bf16 if hardware support
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.00,
        lr_scheduler_type="cosine", #Cosine scheduler to adjust the learning rate over time.
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Train
trainer.train()

Unsloth: Tokenizing ["input_text"] (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 1,386,217,472/8,000,000,000 (17.33% trained)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.97 GiB. GPU 0 has a total capacity of 14.74 GiB of which 462.12 MiB is free. Process 111514 has 14.29 GiB memory in use. Of the allocated memory 13.87 GiB is allocated by PyTorch, and 270.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

# Before running inference, call `FastLanguageModel.for_inference` first

FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    "<|begin_of_text|>Context: Albert Einstein was a theoretical physicist known for his work on relativity. "
    "His contributions revolutionized the understanding of space, time, and gravity. "
    "Question: What is the theory that made Albert Einstein famous? "
    "Answer:"

], return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 25,
    use_cache = True,
)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

# Accumulate the streamed text.
generated_text = ""
for new_text in text_streamer:
    generated_text += new_text

# Post-process the generated text to extract only the answer.
# The answer should starts after "Answer:"
if "Answer:" in generated_text:
    answer_section = generated_text.split("Answer:", 1)[1]
    # If a new question appears, stop there.
    if "Question:" in answer_section:
        answer = answer_section.split("Question:")[0].strip()
    else:
        answer = answer_section.strip()
else:
    answer = generated_text.strip()

# Optionally, wrap the text for display
wrapped_answer = "\n".join(textwrap.wrap(answer, width=max_print_width))
print(wrapped_answer)

Exception in thread Thread-20 (generate):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/peft/peft_model.py", line 1875, in generate
    outputs = self.base_model.generate(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/unsloth/models/llama.py", line 1581, in unsloth_fast_generate
    output = self._old_generate(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 2465, in generate
    result = self._sample(
          

KeyboardInterrupt: 